In [1]:
import edward as ed
import numpy as np
import tensorflow as tf
from edward.models import Normal
import autograd.numpy as np
import autograd.numpy.random as npr
from neuralfingerprint import load_data
from autograd import grad
from keras.layers import Embedding, Dense


Using TensorFlow backend.


### Getting SMILES data

In [2]:
task_params = {'target_name' : 'measured log solubility in mols per litre',
               'data_file'   : 'delaney.csv'}
N_train = 800
N_val   = 20
N_test  = 20

model_params = dict(fp_length=50,    # Usually neural fps need far fewer dimensions than morgan.
                    fp_depth=4,      # The depth of the network equals the fingerprint radius.
                    conv_width=20,   # Only the neural fps need this parameter.
                    h1_size=100,     # Size of hidden layer of network on top of fps.
                    L2_reg=np.exp(-2))

print "Loading data..."
traindata, valdata, testdata = load_data(
    task_params['data_file'], (N_train, N_val, N_test),
    input_name='smiles', target_name=task_params['target_name'])
train_inputs, train_targets = traindata
val_inputs,   val_targets   = valdata
test_inputs,  test_targets  = testdata

print "Task params", task_params

Loading data...
Task params {'target_name': 'measured log solubility in mols per litre', 'data_file': 'delaney.csv'}


In [3]:
from rdkit import Chem

SMILES_CHARS = [' ',
                  '#', '%', '(', ')', '+', '-', '.', '/',
                  '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                  '=', '@',
                  'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P',
                  'R', 'S', 'T', 'V', 'X', 'Z',
                  '[', '\\', ']',
                  'a', 'b', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's',
                  't', 'u']
smi2index = dict( (c,i) for i,c in enumerate( SMILES_CHARS ) )
index2smi = dict( (i,c) for i,c in enumerate( SMILES_CHARS ) )
def smiles_encoder( smiles, maxlen=120 ):
    smiles = Chem.MolToSmiles(Chem.MolFromSmiles( smiles ))
    X = np.zeros( ( maxlen, len( SMILES_CHARS ) ) )
    for i, c in enumerate( smiles ):
        X[i, smi2index[c] ] = 1
    return X
 
def smiles_decoder( X ):
    smi = ''
    X = X.argmax( axis=-1 )
    for i in X:
        smi += index2smi[ i ]
    return smi


In [4]:
X_smiles = train_inputs[0:10]

In [5]:
# Encode and decode for a matrix
mat = np.zeros([len(X_smiles),120,56])
for i in range(0,len(X_smiles)): 
    mat[i,:,:] = smiles_encoder(X_smiles[i])

print( mat.shape )

dec = []
for i in range(0,len(X_smiles)): 
    dec_temp = smiles_decoder(mat[i,:,:])
    dec.append(dec_temp.strip())

print(len(dec))

(10, 120, 56)
10


### RNN / GRU

In [61]:
from edward.models import *
from edward.util import Progbar
H = 5
D = 2
V = 10
E = 2
batch_size = 10
M = 10
nb_steps = 5

N=1000

X_train = np.ones([N, nb_steps ], dtype=np.int32)
y_train = np.ones([N, 1 ], dtype=np.int32)

In [62]:
with tf.variable_scope('model', reuse=True):

    ##### params ######
    Wfo = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
    Wro = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

    Wff = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
    Wrf = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

    Wfy = Normal(loc=tf.zeros([D, H]), scale=tf.ones([D, H]))
    Wry = Normal(loc=tf.zeros([H, H]), scale=tf.ones([H, H]))

    qWfo = Normal(loc=tf.Variable(tf.random_normal([D, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H]))))
    qWro = Normal(loc=tf.Variable(tf.random_normal([H, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H]))))

    qWff = Normal(loc=tf.Variable(tf.random_normal([D, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H]))))
    qWrf = Normal(loc=tf.Variable(tf.random_normal([H, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H]))))

    qWfy = Normal(loc=tf.Variable(tf.random_normal([D, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, H]))))
    qWry = Normal(loc=tf.Variable(tf.random_normal([H, H])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, H]))))
    ####################
                  
    y_ph = tf.placeholder(tf.float32, [batch_size, 1], name='y_ph')

    x = tf.placeholder(tf.int32, [batch_size, nb_steps ], name='x')     

    def gru_cell(hprev, xt):
        #  output gate
        #import pdb; pdb.set_trace()
        i_o = tf.sigmoid(tf.matmul(xt,Wfo) + tf.matmul(hprev,Wro) )
        #  forget gate
        i_f = tf.sigmoid(tf.matmul(xt,Wff) + tf.matmul(hprev,Wrf) )
        #  intermediate
        y = tf.tanh(tf.matmul(xt,Wfy) + tf.matmul( (i_f*hprev),Wry) )
        # new state
        return (1-i_o)*y + (i_o*hprev)
      
    # embed -->
    x_ = Embedding(V, D, name='Embedding')(x)
    
    # initial state of RNN
    h = tf.zeros(shape=(batch_size, H)) # initial state

    for t in range(nb_steps-1):
        h = gru_cell(h, x_[:,t,:])
    print('h : ',h)
    ############# Varitational Inference #########################  
    W1 = Normal(loc=tf.zeros([D, 1]), scale=tf.ones([D, 1]))
    W2 = Normal(loc=tf.zeros([H, D]), scale=tf.ones([H, D]))
    
    qW1 = Normal(loc=tf.Variable(tf.random_normal([D, 1])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, 1]))))
    qW2 = Normal(loc=tf.Variable(tf.random_normal([H, D])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([H, D]))))
    
    def fhw(h_in):
        fhw = tf.matmul(tf.sigmoid(tf.matmul(h_in, W2)), W1)    
        return fhw
    #######################################################   
    y = Normal(loc=fhw(h), scale=0.1 * tf.ones([batch_size,1]))


# inference = ed.KLqp({W1: qW1, W2: qW2, 
#                      Wfo: qWfo, Wro: qWro, 
#                      Wff: qWff, Wrf: qWrf, 
#                      Wfy: qWfy, Wry: qWry}, data={y: y_ph})

inference = ed.KLqp({W1: qW1, W2: qW2}, data={y: y_ph})


('h : ', <tf.Tensor 'model_10/add_15:0' shape=(10, 5) dtype=float32>)


In [63]:
### Run -->
optimizer = tf.train.RMSPropOptimizer(0.01, epsilon=1.0)
inference.initialize(optimizer=optimizer,scale={y: len(X_train) / batch_size}) # always redefine inference before
sess = ed.get_session()
tf.global_variables_initializer().run()

n_epoch = 100
n_iter_per_epoch = 100

for epoch in range(n_epoch):
  avg_loss = 0.0

  pbar = Progbar(n_iter_per_epoch)
  for t in range(1, n_iter_per_epoch + 1):
    pbar.update(t)   
    batch = np.random.randint(0, len(X_train)-1, batch_size)
    info_dict = inference.update({x: X_train[batch], y_ph: y_train[batch]})
    avg_loss  += info_dict['loss']
    
  # Print a lower bound to the average marginal likelihood for an
  # image.
  avg_loss = avg_loss / n_iter_per_epoch
  avg_loss = avg_loss / batch_size
  print("log p(x) >= {:0.3f}".format(avg_loss))

InvalidArgumentError: NodeDef expected inputs 'float, float' do not match 0 inputs specified; Op<name=MatMul; signature=a:T, b:T -> product:T; attr=transpose_a:bool,default=false; attr=transpose_b:bool,default=false; attr=T:type,allowed=[DT_HALF, DT_FLOAT, DT_DOUBLE, DT_INT32, DT_COMPLEX64, DT_COMPLEX128]>; NodeDef: inference_5/sample_5/model_9/MatMul_40315 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false]()

### Testing results

In [41]:
## do test
X_test = np.ones([100, nb_steps])
y_test = np.ones([100, nb_steps])
test1 = sess.run({W1: qW1.sample(),W2: qW2.sample()},{x: X_test})

y_post = ed.copy(y, {W1: qW1,W2: qW2})
y_out = sess.run(y_post, feed_dict={x:X_test})
np.mean(np.square(y_out-y_test))

0.008192643521761696

In [42]:
y_out[0:10]

array([[1.1335912 ],
       [0.9269959 ],
       [0.7887398 ],
       [1.0483382 ],
       [0.9400431 ],
       [1.1055615 ],
       [1.0899656 ],
       [0.9649231 ],
       [0.86622036],
       [1.1573749 ]], dtype=float32)

0.008071045865639342